In [ ]:
import os
import time
import json
import asyncio
import datetime
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import random

import sys
sys.path.append('/home/idies/workspace/Storage/xyu1/persistent/GenCELLAgent/')

from style_utils import load_vgg_model, load_and_preprocess_images, GramMatrix, weighted_style_correlation, load_and_preprocess_png
from src.tools.oneshot_segGPT import seggpt_inference_img


random.seed(1)  # Replace 42 with any fixed number 42, 123, 1234, 12, 1

In [ ]:
def calculate_iou(mask_pred, mask_gt):
    """Compute IoU between two binary masks."""
    intersection = np.logical_and(mask_pred, mask_gt).sum()
    union = np.logical_or(mask_pred, mask_gt).sum()
    if union == 0:
        return 0.0
    return intersection / union

In [ ]:
# Constants
model_path = "/home/idies/workspace/Storage/xyu1/persistent/micro-sam/finetuning/PytorchNeuralStyleTransfer/Models/vgg_conv.pth"
style_layers = ['r11','r21','r31','r41', 'r51']
scales = [1, 1, 1, 1, 1]
#style_weights = [n**2/1e4 for n, s in zip([64,128,256,512,512], scales)]
style_weights = [1.0] * len(style_layers)

vgg = load_vgg_model(model_path)

def extract_features(image_path):
    img_name = os.path.basename(image_path)
    image_tensor = load_and_preprocess_png(os.path.dirname(image_path) + "/", [img_name])[0]
    features = [GramMatrix()(f).detach() for f in vgg(image_tensor, style_layers)]
    return features

# Image and label paths
image_index_list = [15, 25, 33, 37, 53, 65, 82, 89, 9, 97]
random.shuffle(image_index_list)

base_img_path = "/home/idies/workspace/Storage/xyu1/persistent/Langchain/ours_test/test_images/golgi/images/image_{}.png"
base_label_path = "/home/idies/workspace/Storage/xyu1/persistent/Langchain/ours_test/test_images/golgi/labels/label_{}.png"

# Preload features for all images
all_image_paths = [base_img_path.format(idx) for idx in image_index_list]
all_label_paths = [base_label_path.format(idx) for idx in image_index_list]
all_features = [extract_features(path) for path in all_image_paths]

# Store results
reference_selections = []

# For each image as target
for i in range(len(image_index_list)):
    target_index = image_index_list[i]
    target_feature = all_features[i]
    target_image = all_image_paths[i]
    target_label = all_label_paths[i]

    remaining_indices = [j for j in range(len(image_index_list)) if j != i]
    remaining_features = [all_features[j] for j in remaining_indices]
    remaining_paths = [all_image_paths[j] for j in remaining_indices]
    remaining_labels = [all_label_paths[j] for j in remaining_indices]
    remaining_ids = [image_index_list[j] for j in remaining_indices]

    progressive_iou_scores = []

    # Progressive steps
    for k in range(1, len(remaining_indices) + 1):
        candidates = remaining_features[:k]
        candidate_ids = remaining_ids[:k]
        candidate_paths = remaining_paths[:k]
        candidate_labels = remaining_labels[:k]

        step_results = []

        # Compare all k candidates
        for j in range(k):
            score = weighted_style_correlation(candidates[j], target_feature, style_weights)
            step_results.append({
                "reference_index": candidate_ids[j],
                "reference_image": candidate_paths[j],
                "reference_label": candidate_labels[j],
                "score": score
            })

        # Select best one
        best = max(step_results, key=lambda x: x["score"])

        # SegGPT inference
        seggpt_inference_img(target_image, best["reference_image"], best["reference_label"])

        # Paths
        segment_mask_path = "/home/idies/workspace/Storage/xyu1/persistent/Langchain/ours_test/segment_resutls/seggpt_results_mask.png"
        ground_truth_path = target_label

        # Compute IoU
        mask_pred = np.array(Image.open(segment_mask_path).convert("L")) > 128
        mask_gt = np.array(Image.open(ground_truth_path).convert("L")) > 128
        iou_score = calculate_iou(mask_pred, mask_gt)
        progressive_iou_scores.append({
            "num_candidates": k,
            "iou": iou_score,
            "reference_index": best["reference_index"],
            "reference_image": best["reference_image"],
            "reference_label": best["reference_label"],
            "score": best["score"]
        })

    # Store all steps for this target
    reference_selections.append({
        "target_index": target_index,
        "target_image": target_image,
        "target_label": target_label,
        "progressive_iou_scores": progressive_iou_scores
    })
        
# Print all results
for sel in reference_selections:
    print(f"\n=== Target {sel['target_index']} | Progressive IoUs ===")
    for step in sel["progressive_iou_scores"]:
        print(f"Top-{step['num_candidates']} | Ref {step['reference_index']} | Score = {step['score']:.4f} | IoU = {step['iou']:.4f}")
        print(f"  Reference Image: {step['reference_image']}")
        print(f"  Reference Label: {step['reference_label']}")

In [ ]:
import matplotlib.pyplot as plt

# Step 1: Collect IoU values for plotting
progressive_ious_all = []  # shape: [num_targets][num_k_steps]

for sel in reference_selections:
    iou_curve = [step["iou"] for step in sel["progressive_iou_scores"]]
    progressive_ious_all.append(iou_curve)

# Convert to NumPy array for easy averaging
ious_array = np.array(progressive_ious_all)  # shape: (num_targets, num_steps)
average_ious = np.mean(ious_array, axis=0)

# Step 2: Plot curves
plt.figure(figsize=(10, 6))

# Plot per-image curves
for idx, iou_curve in enumerate(ious_array):
    plt.plot(range(1, len(iou_curve)+1), iou_curve, marker='o', label=f'Image {reference_selections[idx]["target_index"]}', alpha=0.5)

# Plot average curve
plt.plot(range(1, len(average_ious)+1), average_ious, marker='o', color='red', linewidth=2.5, label='Average IoU')

# Step 3: Customize and show
plt.title("IoU Progression Over Top-K References")
plt.xlabel("Number of Top-K References")
plt.ylabel("IoU with Ground Truth")
plt.xticks(range(1, len(average_ious)+1))
plt.grid(True)
plt.legend(loc='lower right', bbox_to_anchor=(1.15, 0))
plt.tight_layout()
plt.show()

In [ ]:
average_ious

In [ ]:
np.save("progressive_iou_data_GT_seed_1.npy", reference_selections)
print("Saved to progressive_iou_data_GT_seed_1.npy")

In [ ]:
file_name

In [ ]:
all_avg_iou = []
for seed in ["12","123","1234","42", "ori"]:
    file_name = "progressive_iou_data_GT_seed_"+str(seed)+".npy"
    loaded_data = np.load(file_name, allow_pickle=True)
    loaded_data = loaded_data.tolist()  # Optional: convert back to list
    progressive_ious_all = []  # shape: [num_targets][num_k_steps]

    for sel in loaded_data:
        iou_curve = [step["iou"] for step in sel["progressive_iou_scores"]]
        progressive_ious_all.append(iou_curve)

    # Convert to NumPy array for easy averaging
    ious_array = np.array(progressive_ious_all)  # shape: (num_targets, num_steps)
    average_ious = np.mean(ious_array, axis=0)
    all_avg_iou.append(average_ious)


In [ ]:
all_avg_iou

In [ ]:
# Stack into 2D array
all_avg_iou_array = np.vstack(all_avg_iou)

# Compute mean and std
mean_iou = np.mean(all_avg_iou_array, axis=0)
std_iou = np.std(all_avg_iou_array, axis=0)

# X-axis values (Top-k steps)
x = np.arange(1, len(mean_iou) + 1)

# Plot
plt.figure(figsize=(8, 5))
plt.plot(x, mean_iou, marker='o', label='Mean IoU', color='blue')
plt.fill_between(x, mean_iou - std_iou, mean_iou + std_iou, color='blue', alpha=0.2, label='±1 Std Dev')

# Customize
plt.title("Progressive Reference IoU (Mean ± Std)")
plt.xlabel("Top-K Reference Used")
plt.ylabel("IoU")
plt.xticks(x)
plt.grid(True)
plt.legend()
plt.tight_layout()